In [30]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import numpy as np
import random

def generate_transformed_text(number="3", numberColor=(255,255,255)):
    #Get Large blank canvas
    canvas_size = (500,500)

    
    img = Image.new("RGBA", canvas_size, (0,0,0,0))
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("Freshman.ttf", size=40) 

    num_bbox = draw.textbbox((0,0), number, font=font)
    num_width, num_height = num_bbox[2] - num_bbox[0], num_bbox[3] - num_bbox[1]
    
    num_x = (canvas_size[1] - num_width) // 2
    num_y = (canvas_size[0] - num_height) // 2
    draw.text((num_x, num_y), number, fill=numberColor, font=font)
    
    #Apply affine transform to number
    scale_x = random.uniform(0.6, 1.4)
    scale_y = random.uniform(0.6, 1.4)
    shear_x = random.uniform(-0.4, 0.4)
    shear_y = random.uniform(-0.4, 0.4)
    
    affineMatrix = (scale_x, shear_x, 0, shear_y, scale_y, 0)
    img = img.transform(canvas_size, Image.AFFINE, affineMatrix, resample=Image.BILINEAR)
    
    #Convert to numpy array. Crop image to only take the number itself
    img_data = np.array(img)
    alpha = img_data[:, :, 3]
    nonEmptyPixels = np.where(alpha > 0)
    if nonEmptyPixels[0].size > 0 and nonEmptyPixels[1].size > 0:
        minx, maxx = np.min(nonEmptyPixels[1]), np.max(nonEmptyPixels[1])
        miny, maxy = np.min(nonEmptyPixels[0]), np.max(nonEmptyPixels[0])
        img = img.crop((minx, miny, maxx, maxy))
        return img
    else:
        #print("Error in transform")
        return generate_transformed_text(number)

In [31]:
def add_number_to_jersey(numberImg, jerseyColor):
    numberWidth, numberHeight = numberImg.size
    jerseyWidth, jerseyHeight = 50, 90
    jersey = Image.new('RGB', (jerseyWidth, jerseyHeight), color=jerseyColor)
    #print(jerseyWidth, numberWidth)
    randx = random.randint(0, jerseyWidth - numberWidth)
    randy = random.randint(0, jerseyHeight - numberHeight)
    jersey.paste(numberImg, (randx,randy), numberImg) 
    
    return jersey

In [32]:

def merge(im1: Image.Image, im2: Image.Image) -> Image.Image:
    w = im1.size[0] + im2.size[0]
    h = max(im1.size[1], im2.size[1])
    im = Image.new("RGBA", (w, h))

    im.paste(im1)
    im.paste(im2, (im1.size[0], 0))

    return im

In [33]:
import os
def chooseRandomImage():
    rootImagePath = "imagenet-mini/train"

    #Select random folder (each folder is a different class from imageNet)
    folders = [f for f in os.listdir(rootImagePath) if os.path.isdir(os.path.join(rootImagePath, f))]
    randomFolder = random.choice(folders)
    chosenFolderPath = os.path.join(rootImagePath, randomFolder)

    #Select a random image in that folder
    imageFiles= [f for f in os.listdir(chosenFolderPath) if f.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp'))]
    randomFile = random.choice(imageFiles)
    chosenImagePath = os.path.join(chosenFolderPath, randomFile)

    chosenImage = Image.open(chosenImagePath)
    chosenImageResized = chosenImage.resize((128,100))
    
    return chosenImageResized

def add_background_image(jersey):
    backgroundImage = chooseRandomImage()    
    randx = random.randint(0, backgroundImage.width - jersey.width)
    randy = random.randint(0, backgroundImage.height - jersey.height)

    backgroundImage.paste(jersey, (randx, randy))
    return backgroundImage

In [34]:
def generate_one_image():
    

    
    numberColors = [(240, 240, 240), (10,10,10), (24, 110, 14)]
    numberColor = random.choices(population=numberColors, weights=(0.4, 0.2, 0.4))[0]
    
    #Blue, green, red, darkgray, lightgray
    colors = [(25,50,140), (35, 195, 35), (204,0,0), (16,16,16), (150, 150,150), (176, 172, 56), (240, 240, 240)]
    if numberColor == (240, 240, 240):
       del colors[6]
       del colors[5]
    if numberColor == (10,10,10):
        del colors[3]
    if numberColor == (24,110,14):
        del colors[3]
        del colors[2]
        
    
    colorNoise = (round(random.uniform(-10, 10)), round(random.uniform(-10, 10)), round(random.uniform(-10, 10)))
    jerseyColor = tuple(map(lambda x, y: x + y, random.choice(colors), colorNoise))
    
    digits = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    jerseyNumber1 = random.choice(digits)
    jerseyNumber2 = random.choice(digits)
    
    img1 = generate_transformed_text(jerseyNumber1, numberColor)    
    img2 = generate_transformed_text(jerseyNumber2, numberColor)
    
    jersey1 = add_number_to_jersey(img1, jerseyColor)
    jersey2 = add_number_to_jersey(img2, jerseyColor)
    
    if jerseyNumber1 == '0':
        mergedImage = jersey2
    else:
        mergedImage = merge(jersey1, jersey2)
    
    blur = random.uniform(1,2)
    finalImage = add_background_image(mergedImage).filter(ImageFilter.GaussianBlur(radius = blur))
    
    
    brightness_factor = random.uniform(0.5, 1.5)  # Increase brightness
    enhancer = ImageEnhance.Brightness(finalImage)
    finalImage = enhancer.enhance(brightness_factor)
    
    
    if jerseyNumber1 != "0":
         jerseyNumber = jerseyNumber1 + jerseyNumber2
    else:
        jerseyNumber = jerseyNumber2
    return finalImage, jerseyNumber

finalImage = generate_one_image()
#finalImage[0].show()

In [35]:
numImages = 10000
trueNumbers = []
#In very rare occasions, generate_one_image() will fail. In that case, just try again
for i in range(numImages):
    if i % 100 == 0:
        print(f"Image {i}")
        
    success = False
    while not success:
        try:
            nextImage, nextNumber = generate_one_image()
            trueNumbers.append(nextNumber)
            folder_path = "image/image" + str(i) + ".png"
            nextImage.save(folder_path)
            success = True
        except Exception as e:
            print(f"Error generating image {i}: {e}. Retrying...")
    
# Specify the file path
file_path = 'trueValues.npy'
#Write ground truth to json file
jsonNumbers = {}
for i in range(len(trueNumbers)):
    key = "image"+str(i)
    jsonNumbers[key] = trueNumbers[i]
import json
jsonFile = json.dumps(jsonNumbers)
with open('groundtruth.json', "w") as outfile:
    outfile.write(jsonFile)

Image 0
Error generating image 63: empty range for randrange() (0, -3, -3). Retrying...
Error generating image 71: empty range for randrange() (0, -8, -8). Retrying...
Error generating image 79: empty range for randrange() (0, -1, -1). Retrying...
Error generating image 89: empty range for randrange() (0, -1, -1). Retrying...
Image 100
Error generating image 103: empty range for randrange() (0, -13, -13). Retrying...
Error generating image 107: empty range for randrange() (0, 0, 0). Retrying...
Error generating image 115: empty range for randrange() (0, -11, -11). Retrying...
Error generating image 167: empty range for randrange() (0, -6, -6). Retrying...
Error generating image 170: empty range for randrange() (0, -8, -8). Retrying...
Error generating image 195: empty range for randrange() (0, 0, 0). Retrying...
Image 200
Error generating image 231: empty range for randrange() (0, -3, -3). Retrying...
Error generating image 248: empty range for randrange() (0, -16, -16). Retrying...
Im

In [36]:


jsonNumbers = {}
for i in range(len(trueNumbers)):
    key = "image"+str(i)
    jsonNumbers[key] = trueNumbers[i]

In [37]:
import json
jsonFile = json.dumps(jsonNumbers)
with open('groundtruth.json', "w") as outfile:
    outfile.write(jsonFile)